# Player Acquisition Models
*This notebook is used to train a machine learning model using scikit-learn
and save it in ONNX format.*

## Library imports


In [63]:
import logging
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import GradientBoostingRegressor 

from skl2onnx import to_onnx
import onnxruntime as rt

## Configure Logging

In [64]:
for handler in logging.root.handlers[:]: 
    logging.root.removeHandler(handler)

logging.basicConfig(
    filename='player_acquisition_notebook.log',
    level=logging.INFO,  
)

## Load Data

In [65]:
dataset = pd.read_csv("player_training_data_full.csv")

## Prepare and split data

In [66]:
X = dataset[["waiver_value_tier", "fantasy_regular_season_weeks_remaining", "league_budget_pct_remaining"]]
y = dataset[["winning_bid_dollars"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=0)



## Creating and fitting models

In [67]:
model_10th_percentile = GradientBoostingRegressor(loss="quantile", alpha=0.1)
model_50th_percentile = GradientBoostingRegressor(loss="quantile", alpha=0.5)
model_90th_percentile = GradientBoostingRegressor(loss="quantile", alpha=0.9)

model_10th_percentile.fit(X_train, y_train)
model_50th_percentile.fit(X_train, y_train)
model_90th_percentile.fit(X_train, y_train)


/home/codespace/.local/lib/python3.12/site-packages/sklearn/ensemble/_gb.py:672: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
/home/codespace/.local/lib/python3.12/site-packages/sklearn/ensemble/_gb.py:672: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
/home/codespace/.local/lib/python3.12/site-packages/sklearn/ensemble/_gb.py:672: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


GradientBoostingRegressor(loss='quantile')

## Convert and save these models in ONNX format

In [ ]:
X_array = np.column_stack((X["fantasy_regular_season_weeks_remaining"], X["league_budget_pct_remaining"], X["waiver_value_tier"]))

onx = to_onnx(model_10th_percentile, X_array[:1])
with open("acquisition_model_10_onnx", "wb") as f:
    f.write(onx.SerializeToString())

onx = to_onnx(model_50th_percentile, X_array[:1])
with open("acquisition_model_50_onnx", "wb") as f:
    f.write(onx.SerializeToString())

onx = to_onnx(model_90th_percentile, X_array[:1])
with open("acquisition_model_90_onnx", "wb") as f:
    f.write(onx.SerializeToString())

[[  1  25   3]
 [  1  25   3]
 [  1  25   3]
 [  2  30   1]
 [  2  30   1]
 [  2  30   2]
 [  2  30   2]
 [  3  30   4]
 [  3  30   4]
 [  3  30   4]
 [  4  40   2]
 [  4  40   2]
 [  4  40   4]
 [  4  40   5]
 [  4  40   5]
 [  4  40   5]
 [  5  45   2]
 [  5  45   2]
 [  5  45   4]
 [  6  50   1]
 [  6  50   2]
 [  6  50   2]
 [  6  50   4]
 [  6  50   4]
 [  7  55   2]
 [  7  55   2]
 [  7  55   3]
 [  7  55   3]
 [  8  60   1]
 [  8  60   4]
 [  8  60   2]
 [  8  60   3]
 [  9  65   2]
 [  9  65   1]
 [  9  65   3]
 [  9  65   4]
 [  9  65   5]
 [ 10  70   1]
 [ 10  70   2]
 [ 11  70   1]
 [ 11  70   1]
 [ 11  70   3]
 [ 11  70   4]
 [ 11  70   5]
 [ 11  70   5]
 [ 12  75   2]
 [ 12  75   2]
 [ 12  75   3]
 [ 12  75   4]
 [ 12  75   4]
 [ 12  75   4]
 [ 12  75   5]
 [ 13 100   1]
 [ 13 100   1]
 [ 13 100   1]
 [ 13 100   2]
 [ 13 100   3]
 [ 13 100   3]
 [ 13 100   3]
 [ 13 100   3]
 [ 13 100   3]
 [ 13 100   3]]
